In [1]:
import psycopg2
import sqlalchemy
print("Working!")


Working!


In [7]:
!pip install pandas
import pandas as pd
df = pd.read_csv('customer_shopping_behavior.csv')
df


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly


In [8]:
df.head()
df.info()
df.describe(include='all')
df.isnull().sum()
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(
    lambda x: x.fillna(x.median())
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [4]:
df.isnull().sum()
# Snake case smaller and with _ values
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.columns
df = df.rename(columns = {'purchase_amount_(usd)':'purchase_amount'})
df.columns
# creating the age_group column
labels = ['Young Adult','Adult','middle-aged','Senior']
df['age_group'] = pd.qcut(df['age'],q=4,labels=labels)
df[['age','age_group']].head(10)
df['frequency_of_purchases'].unique()
# create column purchased_frequency_days
frequency_mapping = {
    'Fortnightly':14,
    'Weekly':7,
    'Annually':365,
    'Quarterly':90,
     'Bi-Weekly':14,
    'Monthly':30,
    'Every 3 Months':90

}
df['purchases_frequency_day'] =df['frequency_of_purchases'].map(frequency_mapping)
df[['purchases_frequency_day','frequency_of_purchases']].head()
# Both the columns usses the same data so keep one elimnate one
(df['discount_applied']==df['promo_code_used']).all
df = df.drop('promo_code_used',axis=1)
df.columns


Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'previous_purchases', 'payment_method',
       'frequency_of_purchases', 'age_group', 'purchases_frequency_day'],
      dtype='object')

In [5]:
!pip install psycopg2-binary sqlalchemy
from sqlalchemy import create_engine
username = 'postgres'
password = 'S@a_dd@a_m@7498'
host = 'localhost'
port = '5432'
database = 'customer_behavior'
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

table_name = 'customer'
df.to_sql(table_name,engine,if_exists='replace',index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

OperationalError: (psycopg2.OperationalError) could not translate host name "a_dd@a_m@7498@localhost" to address: Non-recoverable failure in name resolution

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [6]:
from sqlalchemy import create_engine

username = 'postgres'
password = 'S%40a_dd%40a_m%407498'   # encoded password
host = 'localhost'
port = '5432'
database = 'customer_behavior'

engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"
)

df.to_sql("customer", engine, if_exists="replace", index=False)
print("Data successfully loaded!")


Data successfully loaded!


In [ ]:
Q.1. What is the total revenue genrated tb